# Mandatory Challenge
## Context
You work in the data analysis team of a very important company. On Monday, the company shares some good news with you: you just got hired by a major retail company! So, let's get prepared for a huge amount of work!

Then you get to work with your team and define the following tasks to perform:   
1. You need to start your analysis using data from the past.  
2. You need to define a process that takes your daily data as an input and integrates it.  

You are in charge of the second part, so you are provided with a sample file that you will have to read daily. To complete you task, you need the following aggregates:
* One aggregate per store that adds up the rest of the values.
* One aggregate per item that adds up the rest of the values.

You can import the dataset `retail_sales` from Ironhack's database. 

## Your task
Therefore, your process will consist of the following steps:
1. Read the sample file that a daily process will save in your folder. 
2. Clean up the data.
3. Create the aggregates.
4. Write three tables in your local database: 
    - A table for the cleaned data.
    - A table for the aggregate per store.
    - A table for the aggregate per item.

## Instructions
* Read the csv you can find in Ironhack's database.
* Clean the data and create the aggregates as you consider.
* Create the tables in your local database.
* Populate them with your process.

In [1]:
# First step: import 
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
import pandas as pd


In [2]:
# Creating the connection to the database:
driver = 'mysql+pymysql'
ip = '34.65.10.136'
username = 'data-students'
password = 'iR0nH@cK-D4T4B4S3'
db = 'retail_sales'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'

In [3]:
# getting all the data from the table. For this excercise I just call the complete table, but if I had to do it daily, we will write the query to get just the new values generated the day before. 
engine = create_engine(connection_string)
query = 'SELECT * FROM raw_sales'
raw_sales = pd.read_sql(query,engine)
raw_sales

,date,shop_id,item_id,item_price,item_cnt_day
0,2015-01-04,29,1469,1199.0,1.0
1,2015-01-04,28,21364,479.0,1.0
2,2015-01-04,28,21365,999.0,2.0
3,2015-01-04,28,22104,249.0,2.0
4,2015-01-04,28,22091,179.0,1.0
...,...,...,...,...,...
4540,2015-01-04,15,4240,1299.0,1.0
4541,2015-01-04,14,21922,99.0,1.0
4542,2015-01-04,15,1969,3999.0,1.0
4543,2015-01-04,14,22091,179.0,1.0


In [4]:
# CLEANING THE DATA.

# Check the types of data
raw_sales.dtypes


date            datetime64[ns]
shop_id                  int64
item_id                  int64
item_price             float64
item_cnt_day           float64
dtype: object

In [5]:
# We can see that the column 'item_cnt_day' is a float, but we are counting items, so it is not necessary to be a float, we can convert it to an integer. 

raw_sales.astype({'item_cnt_day':'int64'}).dtypes

date            datetime64[ns]
shop_id                  int64
item_id                  int64
item_price             float64
item_cnt_day             int64
dtype: object

In [6]:
# also, the name of this column is not clear, as it seems to be a counting for items per day, but in fact is is a counting for items per day per store. So I will change the name of the column to 'units'
raw_sales.rename(columns = {'item_cnt_day':'units'}, inplace = True)

In [7]:
# Check if there is any null
raw_sales.isnull().sum()

date          0
shop_id       0
item_id       0
item_price    0
units         0
dtype: int64

In [8]:
# in this case, there is no null. If we got one null, as we wouldn't know the real value of it, we will drop the row. We can use the built-in function 'dropna', which automaticaly drops any row with a NaN value.

# I saved into a new dataframe, as this new one will be the 'cleaned table' to upload to the DB. 
cleaned_sales = raw_sales.dropna()


In [9]:
# I upload this cleaned table to a csv file separated by tab.
# cleaned_sales.to_csv('./localdatabase/cleaned_sales.csv', sep='\t', index = False)

In [10]:
# AGGREGATE PER STORE

In [11]:
# First I will create a new column with the revenue per sale, so I will multiply columns 'item_price' by 'units'
cleaned_sales['revenue'] = cleaned_sales['item_price']*cleaned_sales['units']

In [12]:
cleaned_sales.head()

,date,shop_id,item_id,item_price,units,revenue
0,2015-01-04,29,1469,1199.0,1.0,1199.0
1,2015-01-04,28,21364,479.0,1.0,479.0
2,2015-01-04,28,21365,999.0,2.0,1998.0
3,2015-01-04,28,22104,249.0,2.0,498.0
4,2015-01-04,28,22091,179.0,1.0,179.0


In [13]:
# This new table is the one we want, but we just need the 'revenue' column. I will keep the number of units because it's a useful information, but will drop others where the sum does not give any useful information.
sales_by_shop = cleaned_sales.groupby('shop_id').sum().drop(['item_id', 'item_price'], axis = 1)

In [14]:
sales_by_shop.head()

,units,revenue
shop_id,,
2,81.0,103746.0
3,33.0,67443.0
4,39.0,29361.0
5,45.0,33138.0
6,150.0,138678.0


In [15]:
# the units columns has a float number, but we just need an integer, so I will change its datatype.
sales_by_shop['units'] = sales_by_shop['units'].astype('int64')

In [16]:
sales_by_shop.head()

,units,revenue
shop_id,,
2,81,103746.0
3,33,67443.0
4,39,29361.0
5,45,33138.0
6,150,138678.0


In [18]:
# now I will sort by revenue to get the top stores on the top of the table.
sales_by_shop.sort_values(by = 'revenue', ascending = False, inplace = True)

In [19]:
sales_by_shop.head()

,units,revenue
shop_id,,
42,249,330111.0
31,402,304692.0
12,216,295173.0
25,312,288432.0
21,180,228999.0


In [20]:
# Now I have the dataframe ready, I upload it to a csv file separated by tab.
# sales_by_shop.to_csv('./localdatabase/sales_by_shop.csv', sep='\t', index = False)

In [21]:
# AGGREGATE PER ITEM

In [22]:
# Starting with the cleaned_sales with revenue column, I will aggregate by item id, and drop shop id and item price
sales_by_item = cleaned_sales.groupby('item_id').sum().drop(['shop_id', 'item_price'], axis = 1)

In [23]:
sales_by_item.head()

,units,revenue
item_id,,
30,3.0,507.0
31,3.0,1089.0
32,3.0,447.0
42,3.0,897.0
59,3.0,747.0


In [24]:
# changing float to int in units column
sales_by_item['units'] = sales_by_item['units'].astype('int64')

In [25]:
sales_by_item.head()

,units,revenue
item_id,,
30,3,507.0
31,3,1089.0
32,3,447.0
42,3,897.0
59,3,747.0


In [26]:
# now I will sort by revenue to get the top items on the top of the table.
sales_by_item.sort_values(by = 'revenue', ascending = False, inplace = True)

In [27]:
sales_by_item.head()

,units,revenue
item_id,,
1969,66,262134.0
6675,9,242910.0
1971,27,121473.0
1970,12,107988.0
13494,6,89940.0


In [28]:
# Now I have the dataframe ready, I upload it to a csv file separated by tab.
# sales_by_item.to_csv('./localdatabase/sales_by_item.csv', sep='\t', index = False)